<a href="https://colab.research.google.com/github/abiUni/colab/blob/main/Code_suggestions_codeT5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download libraries



In [ ]:
# Install necessary libraries
!pip install datasets
!pip install transformers
!pip install evaluate

import torch
# choose device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install tree-sitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 23.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.6 MB/s eta 0:00

In [ ]:
# Login to Huggingface Hub
#from huggingface_hub import login

#login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Preprocess the data

In [ ]:
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict
from transformers import RobertaTokenizer

# TODO create the whole dataset splits
train_dataset = load_dataset('arefm/suggestions_small_py',use_auth_token=True, split="train").train_test_split(test_size=0.2)
#train_dataset = dataset.select(range(100))
valid_dataset = train_dataset['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_dataset['train'],
    'valid': valid_dataset['test'],
    'test': valid_dataset['train']})

print(dataset)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/arefm___csv/arefm--suggestions_small_py-7cc272b7947463ae/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['id', 'original', 'modified'],
        num_rows: 10243
    })
    valid: Dataset({
        features: ['id', 'original', 'modified'],
        num_rows: 1281
    })
    test: Dataset({
        features: ['id', 'original', 'modified'],
        num_rows: 1280
    })
})


In [ ]:
# loading the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

# setting the model's hyperparameters
max_source_length = 512
max_target_length = 512
# task_prefix helps exploite the knowledge learned by model
task_prefix = "Refine: "

# encoding the training data (inputs, target)
def process_data_to_model_inputs(batch):
  orgnl_code = batch['original'] # original code
  mdfd_code = batch['modified'] # the modified/suggested code 

  # encode the original code sequence
  inputs = [task_prefix + code for code in orgnl_code] 
   
  model_inputs = tokenizer(inputs, max_length=max_source_length, padding="max_length", truncation=True)
  # add data to the same device as the model
  # model_inputs = tokenizer(text, return_tensors="pt").input_ids.to(device)

  # encode the modified code sequence
  labels = tokenizer(mdfd_code, max_length=max_target_length, padding="max_length", truncation=True).input_ids


  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  # length of tokens
  batch["modified_code_len"] = [len(tokenizer(code).input_ids) for code in mdfd_code]
  batch["orig_code_len"] = [len(tokenizer(code).input_ids) for code in inputs]

  return model_inputs


# Prepare the training validation, and test data in batches

dummy_train_100 =  dataset['train'].select(range(100))
dummy_validate_100 = dataset['valid'].select(range(100))

dummy_train = dummy_train_100.map(process_data_to_model_inputs, batched=True, remove_columns=dummy_train_100.column_names)
dummy_validate = dummy_validate_100.map(process_data_to_model_inputs, batched=True, remove_columns=dummy_validate_100.column_names)

filtered_train_dataset_dummy = dummy_train.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length) 
filtered_validation_dataset_dummy = dummy_validate.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length)

#tokenized_train_set = dataset['train'].map(process_data_to_model_inputs, batched=True, remove_columns=dataset['train'].column_names)
#tokenized_validation_set = dataset['valid'].map(process_data_to_model_inputs, batched=True, remove_columns=dataset['valid'].column_names)

# Filter out longer code snippets
#filtered_train_dataset = tokenized_train_set.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length) 
#filtered_validation_dataset = tokenized_validation_set.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length)

print(len(filtered_train_dataset_dummy))
print(len(filtered_validation_dataset_dummy))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1640 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

57
55


# Fine-tune using HuggingFace Trainer

In [ ]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Pre-trained model initialization
if device == 'cuda':
  model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base').cuda()
else:
  model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

# Trainings hyperparameters
batch_size = 8
model_name = 'refine_suggestions_codet5-base' # custom name
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

# Pad the inputs in the batches to max_length
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


# Compute the predictions metrics

In [ ]:
import numpy as np
import evaluate

def postprocess_code(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_simple(eval_preds):
  preds, labels = eval_preds
  print(type(preds))
  return (preds == labels)

def compute_metrics(eval_preds):
  metric = evaluate.load("dvitel/codebleu")
  preds, labels = eval_preds
  if isinstance(preds, tuple):
    preds = preds[0]
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post-processing
  decoded_preds, decoded_labels = postprocess_code(decoded_preds, decoded_labels)

  result = metric.compute(predictions = decoded_preds, references = decoded_labels)
  
  #result = metric.compute(predictions=decoded_preds, references=decoded_labels, lang= 'python')
  #result = {"CodeBLEU": result["score"]}
  #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  #result["gen_len"] = np.mean(prediction_lens)
  #result = {k: round(v, 4) for k, v in result.items()}
  return result  

# Note an optimizer is already defined for us
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=filtered_train_dataset_dummy,
    eval_dataset=filtered_validation_dataset_dummy,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_simple
)

# Start finetuning the model

In [ ]:
trainer.train()
#trainer.save_model(".")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


<class 'numpy.ndarray'>


<ipython-input-28-6f3ec5d52ad7>:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return (preds == labels)


TypeError: ignored